In [1]:
# Ensamble model tuning

In [2]:
#!pip install keras-swa

In [3]:
# !pip install scikit-learn==0.21.3 --user

In [4]:
#!pip install lightgbm

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
import os
import matplotlib.pyplot as plt

In [2]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import lightgbm as lgbm

Using TensorFlow backend.


In [3]:
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, GaussianNoise, Concatenate, LeakyReLU
from keras.models import Model, load_model
from keras import optimizers, callbacks
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint,Callback, EarlyStopping
from keras import backend as K
from swa.keras import SWA # swa


# 데이터 불러오기

In [1]:
# 0. 패키지 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. 데이터 불러오기
train = pd.read_csv('train.csv', index_col=0) # 코랩 사용 시 경로(/content/drive/My Drive/Colab Notebooks/) 추가
test = pd.read_csv('test.csv', index_col=0)
sample_submission = pd.read_csv('sample_submission.csv', index_col=0)

# 2. Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# 3.모델에 적용할 데이터 셋 준비 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler, Normalizer

train_x = train.drop(columns=['type', 'type_num'], axis=1)
del train_x['fiberID']
train_y = train['type_num']
test_x = test
del test_x['fiberID']

x = train_x.copy() 

down_quantiles = x.quantile(0.002) # 전처리 비율 조정
up_quantiles = x.quantile(0.998)

outliers_low = (x < down_quantiles)
outliers_high = (x > up_quantiles)

x[outliers_low] = np.nan
x.fillna(down_quantiles, inplace=True)

x[outliers_high] = np.nan
x.fillna(up_quantiles, inplace=True)

In [3]:
train_y

id
0          8
1          8
2          8
3          8
4         10
5         18
6          8
7          5
8          8
9          1
10         8
11         8
12         7
13         5
14         5
15         8
16        18
17        18
18         6
19        18
20        15
21         4
22         5
23         8
24        18
25         1
26        10
27         8
28         6
29         6
          ..
199961     6
199962    14
199963     8
199964     5
199965     8
199966    13
199967    15
199968     6
199969     0
199970    10
199971    14
199972     8
199973     6
199974    18
199975     8
199976    15
199977     6
199978    13
199979     4
199980     8
199981     8
199982     3
199983    15
199984    18
199985     7
199986     8
199987     6
199988    10
199989     6
199990     8
Name: type_num, Length: 199991, dtype: int64

In [5]:
x = x.values
train_y = train_y.values

x_train, x_test, y_train, y_test = train_test_split(x, train_y, stratify=train_y, train_size=0.9)

# # 3-1 다중 분류 시 One-hot Encoding
# from sklearn.preprocessing import OneHotEncoder
# onehot_encoder = OneHotEncoder()
# y_train = y_train.reshape(len(y_train), -1) 
# y_val = y_val.reshape(len(y_val), -1)
# y_train = onehot_encoder.fit_transform(y_train)
# y_val = onehot_encoder.fit_transform(y_val)

# print(x_train.shape)
# print(y_train.shape)

In [6]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
import tensorflow as tf

class Gelu(Activation):
    def __init__(self, activation, **kwargs):
        super(Gelu, self).__init__(activation, **kwargs)
        self.__name__='gelu'
        
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * np.power(x, 3))))

get_custom_objects().update({'gelu': Gelu(gelu)})

# Scaler

In [7]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# scaler1=RobustScaler()
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

test_x =scaler.transform(test_x)

In [8]:
print(x_train.shape)
print(y_train.shape)

(179991, 20)
(179991,)


# Variables

In [9]:
drop1 = 0.4
drop2 = 0.45
drop3 = 0.5

In [10]:
import datetime
start=datetime.datetime.now()
print(start)

2020-02-24 09:34:44.794129


# Deep Learning Model
- EarlyStop, CosineScheduler, CheckPoint

In [11]:
def model_1(epochs=1200, drop=0.45):

    inps= Input(shape = (20,))
    
    la = Dense(128)(inps)
    la = Dense(256, activation='relu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='relu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='relu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='relu')(la)
    la = Dropout(drop)(la)

    outs=Dense(19, activation='softmax',name='output')(la)

    models = Model(inputs=inps, outputs=outs)
    models.summary()
    models.compile(loss="sparse_categorical_crossentropy", \
                   optimizer=Adam(lr=1.e-4, beta_1=0.9, beta_2=0.98, epsilon=1e-08, decay=1e-6),metrics=['accuracy'])
    
    return models


In [12]:
def model_2(epochs=1200, drop=0.45):

    inps= Input(shape = (20,))
    
    la = Dense(128)(inps)
    la = Dense(256, activation='elu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='elu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='elu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='elu')(la)
    la = Dropout(drop)(la)
    outs=Dense(19, activation='softmax',name='output')(la)

    models = Model(inputs=inps, outputs=outs)
    models.summary()
    models.compile(loss="sparse_categorical_crossentropy", \
                   optimizer=Adam(lr=1.e-4, beta_1=0.9, beta_2=0.98, epsilon=1e-08, decay=1e-6),metrics=['accuracy'])
    
    return models


In [13]:
def model_3(epochs=1200, drop=0.45):

    inps= Input(shape = (20,))
    
    la = Dense(128)(inps)
    la = Dense(256, activation='selu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='selu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='selu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='selu')(la)
    la = Dropout(drop)(la)
    outs=Dense(19, activation='softmax',name='output')(la)

    models = Model(inputs=inps, outputs=outs)
    models.summary()
    models.compile(loss="sparse_categorical_crossentropy", \
                   optimizer=Adam(lr=1.e-4, beta_1=0.9, beta_2=0.98, epsilon=1e-08, decay=1e-6),metrics=['accuracy'])
    
    return models


In [14]:
def model_4(epochs=1200, drop=0.45):

    inps= Input(shape = (20,))
    
    la = Dense(128)(inps)
    la = Dense(256)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop)(la)
    outs=Dense(19, activation='softmax',name='output')(la)

    models = Model(inputs=inps, outputs=outs)
    models.summary()
    models.compile(loss="sparse_categorical_crossentropy", \
                   optimizer=Adam(lr=1.e-4, beta_1=0.9, beta_2=0.98, epsilon=1e-08, decay=1e-6),metrics=['accuracy'])
    
    return models

In [15]:
def model_5(epochs=1200, drop=0.45):

    inps= Input(shape = (20,))
    
    la = Dense(128)(inps)
    la = LeakyReLU(alpha=0.07)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = LeakyReLU(alpha=0.07)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = LeakyReLU(alpha=0.07)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = LeakyReLU(alpha=0.07)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = Dropout(drop)(la)
    outs=Dense(19, activation='softmax',name='output')(la)

    models = Model(inputs=inps, outputs=outs)
    models.summary()
    models.compile(loss="sparse_categorical_crossentropy", \
                   optimizer=Adam(lr=1.e-4, beta_1=0.9, beta_2=0.98, epsilon=1e-08, decay=1e-6),metrics=['accuracy'])
    
    return models

In [16]:
sk_params = { 'validation_split': 1./9. }
nepoch1 = 1000
nepoch2 = 850
nepoch3 = 900

model_1a = KerasClassifier(build_fn = model_1, epochs = nepoch1, drop=drop1, verbose=2, batch_size=256, **sk_params)

model_1b = KerasClassifier(build_fn = model_1, epochs = nepoch2, drop=drop2, verbose=2, batch_size=256, **sk_params)

model_1c = KerasClassifier(build_fn = model_1, epochs = nepoch3, drop=drop3, verbose=2, batch_size=256, **sk_params)

model_2a = KerasClassifier(build_fn = model_2, epochs = nepoch1, drop=drop1, verbose=2, batch_size=256, **sk_params)

model_2b = KerasClassifier(build_fn = model_2, epochs = nepoch2, drop=drop2, verbose=2, batch_size=256, **sk_params)

model_2c = KerasClassifier(build_fn = model_2, epochs = nepoch3, drop=drop3, verbose=2, batch_size=256, **sk_params)

model_3a = KerasClassifier(build_fn = model_3, epochs = nepoch1, drop=drop1, verbose=2, batch_size=256, **sk_params)

model_3b = KerasClassifier(build_fn = model_3, epochs = nepoch2, drop=drop2, verbose=2, batch_size=256, **sk_params)

model_3c = KerasClassifier(build_fn = model_3, epochs = nepoch3, drop=drop3, verbose=2, batch_size=256, **sk_params)

model_4a = KerasClassifier(build_fn = model_4, epochs = nepoch1, drop=drop1, verbose=2, batch_size=256, **sk_params)

model_4b = KerasClassifier(build_fn = model_4, epochs = nepoch2, drop=drop2, verbose=2, batch_size=256, **sk_params)

model_4c = KerasClassifier(build_fn = model_4, epochs = nepoch3, drop=drop3, verbose=2, batch_size=256, **sk_params)

model_5a = KerasClassifier(build_fn = model_5, epochs = nepoch1, drop=drop1, verbose=2, batch_size=256, **sk_params)

model_5b = KerasClassifier(build_fn = model_5, epochs = nepoch2, drop=drop2, verbose=2, batch_size=256, **sk_params)

model_5c = KerasClassifier(build_fn = model_5, epochs = nepoch3, drop=drop3, verbose=2, batch_size=256, **sk_params)



In [28]:
ensemble_clf = VotingClassifier(estimators = [('model6', model_2a), ('model7', model_2b), ('model8', model_2c), 
                                             ('model9', model_3a), ('model10', model_3b), ('model11', model_3c)], voting = 'soft')

# ensemble_clf = VotingClassifier(estimators = [ ('model1', model_1a), ],  voting = 'soft')

In [ ]:
ensemble_clf.fit(x_train, y_train)

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 20)                0         
_________________________________________________________________
dense_65 (Dense)             (None, 128)               2688      
_________________________________________________________________
dense_66 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_55 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_67 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_56 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_68 (Dense)             (None, 256)               657

 - 2s - loss: 0.4642 - accuracy: 0.8434 - val_loss: 0.4194 - val_accuracy: 0.8573
Epoch 68/1000
 - 2s - loss: 0.4635 - accuracy: 0.8429 - val_loss: 0.4169 - val_accuracy: 0.8571
Epoch 69/1000
 - 2s - loss: 0.4633 - accuracy: 0.8435 - val_loss: 0.4154 - val_accuracy: 0.8579
Epoch 70/1000
 - 2s - loss: 0.4634 - accuracy: 0.8433 - val_loss: 0.4205 - val_accuracy: 0.8564
Epoch 71/1000
 - 2s - loss: 0.4610 - accuracy: 0.8437 - val_loss: 0.4209 - val_accuracy: 0.8566
Epoch 72/1000
 - 2s - loss: 0.4614 - accuracy: 0.8440 - val_loss: 0.4192 - val_accuracy: 0.8576
Epoch 73/1000
 - 2s - loss: 0.4594 - accuracy: 0.8446 - val_loss: 0.4171 - val_accuracy: 0.8568
Epoch 74/1000
 - 2s - loss: 0.4596 - accuracy: 0.8443 - val_loss: 0.4178 - val_accuracy: 0.8570
Epoch 75/1000
 - 2s - loss: 0.4587 - accuracy: 0.8440 - val_loss: 0.4123 - val_accuracy: 0.8582
Epoch 76/1000
 - 2s - loss: 0.4586 - accuracy: 0.8443 - val_loss: 0.4133 - val_accuracy: 0.8577
Epoch 77/1000
 - 2s - loss: 0.4566 - accuracy: 0.8456 

Epoch 152/1000
 - 2s - loss: 0.4318 - accuracy: 0.8523 - val_loss: 0.3976 - val_accuracy: 0.8619
Epoch 153/1000
 - 2s - loss: 0.4313 - accuracy: 0.8529 - val_loss: 0.3919 - val_accuracy: 0.8623
Epoch 154/1000
 - 2s - loss: 0.4322 - accuracy: 0.8524 - val_loss: 0.3961 - val_accuracy: 0.8628
Epoch 155/1000
 - 2s - loss: 0.4298 - accuracy: 0.8539 - val_loss: 0.3937 - val_accuracy: 0.8628
Epoch 156/1000
 - 2s - loss: 0.4305 - accuracy: 0.8529 - val_loss: 0.3973 - val_accuracy: 0.8625
Epoch 157/1000
 - 2s - loss: 0.4292 - accuracy: 0.8536 - val_loss: 0.3941 - val_accuracy: 0.8623
Epoch 158/1000
 - 2s - loss: 0.4288 - accuracy: 0.8530 - val_loss: 0.3993 - val_accuracy: 0.8631
Epoch 159/1000
 - 2s - loss: 0.4287 - accuracy: 0.8531 - val_loss: 0.3959 - val_accuracy: 0.8630
Epoch 160/1000
 - 2s - loss: 0.4296 - accuracy: 0.8535 - val_loss: 0.3985 - val_accuracy: 0.8625
Epoch 161/1000
 - 2s - loss: 0.4294 - accuracy: 0.8533 - val_loss: 0.4021 - val_accuracy: 0.8605
Epoch 162/1000
 - 2s - loss: 0

Epoch 237/1000
 - 2s - loss: 0.4176 - accuracy: 0.8568 - val_loss: 0.3853 - val_accuracy: 0.8640
Epoch 238/1000
 - 2s - loss: 0.4187 - accuracy: 0.8566 - val_loss: 0.3884 - val_accuracy: 0.8657
Epoch 239/1000
 - 2s - loss: 0.4180 - accuracy: 0.8572 - val_loss: 0.3885 - val_accuracy: 0.8652
Epoch 240/1000
 - 2s - loss: 0.4181 - accuracy: 0.8573 - val_loss: 0.3924 - val_accuracy: 0.8644
Epoch 241/1000
 - 2s - loss: 0.4180 - accuracy: 0.8577 - val_loss: 0.3859 - val_accuracy: 0.8654
Epoch 242/1000
 - 2s - loss: 0.4180 - accuracy: 0.8573 - val_loss: 0.3879 - val_accuracy: 0.8650
Epoch 243/1000
 - 2s - loss: 0.4179 - accuracy: 0.8575 - val_loss: 0.3875 - val_accuracy: 0.8661
Epoch 244/1000
 - 2s - loss: 0.4195 - accuracy: 0.8568 - val_loss: 0.3902 - val_accuracy: 0.8666
Epoch 245/1000
 - 2s - loss: 0.4177 - accuracy: 0.8576 - val_loss: 0.3875 - val_accuracy: 0.8651
Epoch 246/1000
 - 2s - loss: 0.4168 - accuracy: 0.8580 - val_loss: 0.3860 - val_accuracy: 0.8661
Epoch 247/1000
 - 2s - loss: 0

Epoch 322/1000
 - 2s - loss: 0.4118 - accuracy: 0.8588 - val_loss: 0.3844 - val_accuracy: 0.8657
Epoch 323/1000
 - 2s - loss: 0.4114 - accuracy: 0.8589 - val_loss: 0.3853 - val_accuracy: 0.8653
Epoch 324/1000
 - 2s - loss: 0.4118 - accuracy: 0.8588 - val_loss: 0.3836 - val_accuracy: 0.8647
Epoch 325/1000
 - 2s - loss: 0.4109 - accuracy: 0.8595 - val_loss: 0.3839 - val_accuracy: 0.8669
Epoch 326/1000
 - 2s - loss: 0.4111 - accuracy: 0.8595 - val_loss: 0.3894 - val_accuracy: 0.8632
Epoch 327/1000
 - 2s - loss: 0.4121 - accuracy: 0.8587 - val_loss: 0.3831 - val_accuracy: 0.8661
Epoch 328/1000
 - 2s - loss: 0.4113 - accuracy: 0.8595 - val_loss: 0.3850 - val_accuracy: 0.8653
Epoch 329/1000
 - 2s - loss: 0.4109 - accuracy: 0.8592 - val_loss: 0.3816 - val_accuracy: 0.8676
Epoch 330/1000
 - 2s - loss: 0.4094 - accuracy: 0.8589 - val_loss: 0.3834 - val_accuracy: 0.8669
Epoch 331/1000
 - 2s - loss: 0.4098 - accuracy: 0.8602 - val_loss: 0.3841 - val_accuracy: 0.8664
Epoch 332/1000
 - 2s - loss: 0

...

Epoch 537/900
 - 2s - loss: 0.4419 - accuracy: 0.8520 - val_loss: 0.3964 - val_accuracy: 0.8625
Epoch 538/900
 - 2s - loss: 0.4407 - accuracy: 0.8529 - val_loss: 0.3973 - val_accuracy: 0.8640
Epoch 539/900
 - 2s - loss: 0.4387 - accuracy: 0.8534 - val_loss: 0.3936 - val_accuracy: 0.8641
Epoch 540/900
 - 2s - loss: 0.4396 - accuracy: 0.8532 - val_loss: 0.3969 - val_accuracy: 0.8631
Epoch 541/900
 - 2s - loss: 0.4397 - accuracy: 0.8525 - val_loss: 0.3998 - val_accuracy: 0.8600
Epoch 542/900
 - 2s - loss: 0.4409 - accuracy: 0.8528 - val_loss: 0.3986 - val_accuracy: 0.8621
Epoch 543/900
 - 2s - loss: 0.4414 - accuracy: 0.8518 - val_loss: 0.3981 - val_accuracy: 0.8630
Epoch 544/900
 - 2s - loss: 0.4405 - accuracy: 0.8529 - val_loss: 0.3977 - val_accuracy: 0.8636
Epoch 545/900
 - 2s - loss: 0.4411 - accuracy: 0.8521 - val_loss: 0.3942 - val_accuracy: 0.8635
Epoch 546/900
 - 2s - loss: 0.4397 - accuracy: 0.8527 - val_loss: 0.3943 - val_accuracy: 0.8623
Epoch 547/900
 - 2s - loss: 0.4392 - acc

Epoch 623/900
 - 2s - loss: 0.4373 - accuracy: 0.8530 - val_loss: 0.3975 - val_accuracy: 0.8625
Epoch 624/900
 - 2s - loss: 0.4374 - accuracy: 0.8531 - val_loss: 0.3952 - val_accuracy: 0.8643
Epoch 625/900
 - 2s - loss: 0.4385 - accuracy: 0.8539 - val_loss: 0.3949 - val_accuracy: 0.8642
Epoch 626/900
 - 2s - loss: 0.4383 - accuracy: 0.8529 - val_loss: 0.3982 - val_accuracy: 0.8627
Epoch 627/900
 - 2s - loss: 0.4378 - accuracy: 0.8527 - val_loss: 0.3946 - val_accuracy: 0.8638
Epoch 628/900
 - 2s - loss: 0.4381 - accuracy: 0.8530 - val_loss: 0.3934 - val_accuracy: 0.8653
Epoch 629/900
 - 2s - loss: 0.4367 - accuracy: 0.8523 - val_loss: 0.3927 - val_accuracy: 0.8635
Epoch 630/900
 - 2s - loss: 0.4365 - accuracy: 0.8544 - val_loss: 0.3953 - val_accuracy: 0.8624
Epoch 631/900
 - 2s - loss: 0.4368 - accuracy: 0.8528 - val_loss: 0.3947 - val_accuracy: 0.8627
Epoch 632/900
 - 2s - loss: 0.4380 - accuracy: 0.8525 - val_loss: 0.3969 - val_accuracy: 0.8633
Epoch 633/900
 - 2s - loss: 0.4377 - acc

Epoch 709/900
 - 2s - loss: 0.4359 - accuracy: 0.8539 - val_loss: 0.3951 - val_accuracy: 0.8644
Epoch 710/900
 - 2s - loss: 0.4366 - accuracy: 0.8535 - val_loss: 0.3959 - val_accuracy: 0.8628
Epoch 711/900
 - 2s - loss: 0.4347 - accuracy: 0.8540 - val_loss: 0.3949 - val_accuracy: 0.8636
Epoch 712/900
 - 2s - loss: 0.4358 - accuracy: 0.8539 - val_loss: 0.3931 - val_accuracy: 0.8637
Epoch 713/900
 - 2s - loss: 0.4350 - accuracy: 0.8532 - val_loss: 0.3924 - val_accuracy: 0.8659
Epoch 714/900
 - 2s - loss: 0.4349 - accuracy: 0.8537 - val_loss: 0.3936 - val_accuracy: 0.8635
Epoch 715/900
 - 2s - loss: 0.4358 - accuracy: 0.8545 - val_loss: 0.3925 - val_accuracy: 0.8650
Epoch 716/900
 - 2s - loss: 0.4345 - accuracy: 0.8545 - val_loss: 0.3926 - val_accuracy: 0.8640
Epoch 717/900
 - 2s - loss: 0.4352 - accuracy: 0.8538 - val_loss: 0.3970 - val_accuracy: 0.8631
Epoch 718/900
 - 2s - loss: 0.4351 - accuracy: 0.8540 - val_loss: 0.3956 - val_accuracy: 0.8634
Epoch 719/900
 - 2s - loss: 0.4357 - acc

Epoch 795/900
 - 2s - loss: 0.4341 - accuracy: 0.8541 - val_loss: 0.3928 - val_accuracy: 0.8646
Epoch 796/900
 - 2s - loss: 0.4353 - accuracy: 0.8537 - val_loss: 0.3950 - val_accuracy: 0.8646
Epoch 797/900
 - 2s - loss: 0.4344 - accuracy: 0.8538 - val_loss: 0.3918 - val_accuracy: 0.8651
Epoch 798/900
 - 2s - loss: 0.4331 - accuracy: 0.8544 - val_loss: 0.3982 - val_accuracy: 0.8629
Epoch 799/900
 - 2s - loss: 0.4342 - accuracy: 0.8535 - val_loss: 0.3941 - val_accuracy: 0.8636
Epoch 800/900
 - 2s - loss: 0.4326 - accuracy: 0.8547 - val_loss: 0.3944 - val_accuracy: 0.8632
Epoch 801/900
 - 2s - loss: 0.4336 - accuracy: 0.8552 - val_loss: 0.3946 - val_accuracy: 0.8608
Epoch 802/900
 - 2s - loss: 0.4332 - accuracy: 0.8547 - val_loss: 0.3903 - val_accuracy: 0.8645
Epoch 803/900
 - 2s - loss: 0.4342 - accuracy: 0.8535 - val_loss: 0.3926 - val_accuracy: 0.8640
Epoch 804/900
 - 2s - loss: 0.4340 - accuracy: 0.8537 - val_loss: 0.3927 - val_accuracy: 0.8649
Epoch 805/900
 - 2s - loss: 0.4336 - acc

Epoch 881/900
 - 2s - loss: 0.4338 - accuracy: 0.8544 - val_loss: 0.3937 - val_accuracy: 0.8636
Epoch 882/900
 - 2s - loss: 0.4319 - accuracy: 0.8545 - val_loss: 0.3932 - val_accuracy: 0.8627
Epoch 883/900
 - 2s - loss: 0.4338 - accuracy: 0.8548 - val_loss: 0.3918 - val_accuracy: 0.8642
Epoch 884/900
 - 2s - loss: 0.4318 - accuracy: 0.8550 - val_loss: 0.3920 - val_accuracy: 0.8641
Epoch 885/900
 - 2s - loss: 0.4323 - accuracy: 0.8546 - val_loss: 0.3945 - val_accuracy: 0.8636
Epoch 886/900
 - 2s - loss: 0.4329 - accuracy: 0.8543 - val_loss: 0.3938 - val_accuracy: 0.8635
Epoch 887/900
 - 2s - loss: 0.4324 - accuracy: 0.8549 - val_loss: 0.3907 - val_accuracy: 0.8638
Epoch 888/900
 - 2s - loss: 0.4323 - accuracy: 0.8544 - val_loss: 0.3916 - val_accuracy: 0.8648
Epoch 889/900
 - 2s - loss: 0.4336 - accuracy: 0.8553 - val_loss: 0.3914 - val_accuracy: 0.8649
Epoch 890/900
 - 2s - loss: 0.4303 - accuracy: 0.8556 - val_loss: 0.3925 - val_accuracy: 0.8633
Epoch 891/900
 - 2s - loss: 0.4328 - acc

In [24]:
print(ensemble_clf.score(x_test, y_test))

0.873


In [25]:
end=datetime.datetime.now()
print("걸린 시간:", end-start)

pred_test=ensemble_clf.predict_proba(test_x)

걸린 시간: 8:48:20.271172


In [26]:
print(pred_test.shape)

(10009, 19)


# Model Predict

In [27]:
# 제출 파일 생성
submission = pd.DataFrame(data=pred_test, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('submission_data_KerasClassifier_2.csv', index=True)